# Data Preprocessing

In [114]:
import pandas as pd
import numpy as np

In [115]:
# Load the data
df = pd.read_csv('/Users/vanessahuang/Desktop/BI_capstone/信義房屋/data/0326(初版).csv')

### 處理 missing values

In [116]:
# Check if there are any missing values
missing_value = df.isnull().sum()
missing_value = missing_value.reset_index()
missing_value.columns = ['column_name', 'missing_values']
missing_value.to_csv('data/缺失值.csv', index=False)

In [117]:
# 移除「備註」欄位
df = df.drop(columns=['備註', '土地位置建物門牌', '物件編號', '移轉層次'])
# 「都市土地使用分區」有缺值部分直接刪掉整條 row
df = df.dropna(subset=['都市土地使用分區', '主要建材'])
print(df.shape)

(48441, 92)


### Encoding Categorical Variables

需要處理的欄位：都市土地使用分區、主要建材

> **都市土地使用分區**

In [118]:
land_distribution = df['都市土地使用分區'].value_counts()
land_distribution = land_distribution.reset_index()
land_distribution.columns = ['land_usage', 'count']
land_distribution.to_csv('data/都市土地使用分區.csv', index=False)

In [119]:
# 把開頭是「都市：其他」的資料先全部變成一個種類：「都市」
df['都市土地使用分區'] = df['都市土地使用分區'].apply(lambda x: '都市' if x.startswith('都市：其他:') else x)
df['都市土地使用分區'].value_counts()

住     28315
商     15655
都市     3581
工       884
農         6
Name: 都市土地使用分區, dtype: int64

In [120]:
# 做 one-hot encoding
df_encoded = pd.get_dummies(df, columns=['都市土地使用分區'], drop_first=True)
df_encoded.head()

,土地移轉總面積平方公尺,交易年月日,總樓層數,主要建材,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,...,車位類別_坡道機械,車位類別_塔式車位,土地,建物,車位,建物年齡,都市土地使用分區_商,都市土地使用分區_工,都市土地使用分區_農,都市土地使用分區_都市
0,29.50,1100101,4,鋼筋混凝土加強磚造,86.21,3,2,1,1,0,...,0,0,1,1,0,18701,0,0,0,0
2,10.84,1100101,12,鋼筋混凝土造,82.75,2,1,0,1,1,...,0,0,2,1,0,15687,0,0,0,0
3,27.35,1100101,12,鋼筋混凝土造,169.77,1,1,1,1,1,...,0,0,1,2,0,11826,1,0,0,0
4,2.35,1100101,14,鋼筋混凝土造,22.61,1,1,1,1,1,...,0,0,1,1,0,14589,1,0,0,0
5,28.50,1100101,5,鋼筋混凝土造,132.08,3,3,2,1,0,...,0,0,1,1,0,13181,0,0,0,0


> **主要建材**

In [121]:
main_materials = df_encoded['主要建材'].value_counts()
main_materials = main_materials.reset_index()
main_materials.columns = ['main_materials', 'count']
main_materials.to_csv('data/主要建材.csv', index=False)

In [122]:
# 將 count 少於 100 的主要建材歸類為「其他」
material_counts = df_encoded['主要建材'].value_counts()
mask = material_counts < 100
materials_to_replace = mask[mask].index.tolist()
df_encoded['主要建材'] = df_encoded['主要建材'].apply(lambda x: '其他' if x in materials_to_replace else x)

# Check the updated value counts
print(df_encoded['主要建材'].value_counts())

鋼筋混凝土造      43615
鋼骨造          1505
加強磚造         1503
鋼骨鋼筋混凝土造     1410
其他            408
Name: 主要建材, dtype: int64


In [123]:
# 做 one-hot encoding
data = pd.get_dummies(df_encoded, columns=['主要建材'], drop_first=True)
data.head()

,土地移轉總面積平方公尺,交易年月日,總樓層數,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,建物現況格局-隔間,有無管理組織,總價元,...,車位,建物年齡,都市土地使用分區_商,都市土地使用分區_工,都市土地使用分區_農,都市土地使用分區_都市,主要建材_加強磚造,主要建材_鋼筋混凝土造,主要建材_鋼骨造,主要建材_鋼骨鋼筋混凝土造
0,29.50,1100101,4,86.21,3,2,1,1,0,11500000,...,0,18701,0,0,0,0,0,0,0,0
2,10.84,1100101,12,82.75,2,1,0,1,1,18500000,...,0,15687,0,0,0,0,0,1,0,0
3,27.35,1100101,12,169.77,1,1,1,1,1,19000000,...,0,11826,1,0,0,0,0,1,0,0
4,2.35,1100101,14,22.61,1,1,1,1,1,4000000,...,0,14589,1,0,0,0,0,1,0,0
5,28.50,1100101,5,132.08,3,3,2,1,0,35000000,...,0,13181,0,0,0,0,0,1,0,0


### Splitting the data (test / train)
- 業主：交易日期 110 -112 年用來當Training，先確認該時間段的資料筆數，若是足夠的話可以用日期去切就好了
- 但我看不懂他的意思，所以我用 20-80 切

In [124]:
# First ensure the transaction date is an integer or string
data['交易年月日'] = data['交易年月日'].astype(str)

# Sort the DataFrame based on the transaction date
data_sorted = data.sort_values(by='交易年月日')

# Calculate the index at which to split the data (80% for training, 20% for testing)
split_index = int(len(data_sorted) * 0.8)

# Split the DataFrame into training and testing sets
training_data = data_sorted[:split_index]
test_data = data_sorted[split_index:]

# Verify the split
print(f"Training data shape: {training_data.shape}")
print(f"Test data shape: {test_data.shape}")

Training data shape: (38752, 98)
Test data shape: (9689, 98)
